## [미니프로젝트] 악성사이트 탐지 머신러닝 모델 개발

### 여러분은 기업 보안팀에서 근무중인 엔지니어로써, 웹페이지에서 추출한 Feature(특징) 기반으로 악성사이트를 탐지하는 머신러닝 모델 개발 미션을 부여받았습니다.

### ▣ 우리가 풀어야 하는 문제는 무엇인가요?
 - 웹 페이지에서 Feature를 추출하세요.
 - 악성사이트 여부를 판별하는 성능 좋은 AI모델을 생성하세요.

<br>

---

## ▣ 데이터 소개
* 웹 크롤링 데이터셋 : Feature_Website.xlsx

## ▣ 웹 크롤링 데이터셋의 변수 소개
* html_code : 크롤링을 활용해 수집한 HTML Code 원본
* repu : 악성사이트 여부 (malicious : 악성사이트, benign : 정상사이트)
<br>

---

## <b>[1단계] 데이터 수집</b>

* 1단계에서는 크롤링으로 수집한 HTML Code를 활용해 Feature를 만드는 과정을 체험합니다.

# <b>Step 0. 본격적인 실습 전 packages 설치
* Beautifulsoup 라이브러리 설치
* openpyxl 라이브러리 설치

In [52]:
from bs4 import BeautifulSoup
import openpyxl 


* 데이터 프레임 관련 라이브러리 Import

In [53]:
import pandas as pd
import numpy as np

---
## <b>Q1. 데이터 불러오기
### 정상/악성 HTML Code가 저장된 엑셀파일 불러오기
- 파일명 : Feature Website.xlsx


### <span style="color:darkred">[문제1] Pandas 라이브러리를 활용해서 'Feature Website.xlsx'파일을 'df' 변수에 저장하고 그 info()및 head()를 통해 데이터를 확인하세요.<span>

In [54]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df = pd.read_excel("C:/Users/admin/Desktop/에이블 실습/miniproject2/실습파일/Feature_Website.xlsx", engine = 'openpyxl')

In [55]:
# 데이터 프레임의 info를 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   html_code  40 non-null     object
 1   repu       40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [56]:
# 불러온 데이터를 확인합니다.
df.head()

,html_code,repu
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious


---
# <b>Step 1. 데이터 수집

### 주어진 데이터로만 모델링 하는 경우는 드뭅니다.
### 주어진 데이터 외 추가로 데이터를 수집 또는 생성해야 하는 경우가 많습니다.
### 이번 과정에서는 웹 크롤러를 통해 수집된 정상/악성 사이트 HTML 데이터에서
### BeatifulSoup 라이브러리를 활용 필요한 Feature(특징)를 추출해 보도록 하겠습니다.
### 정상/악성 사이트 HTML Code는 사전에 수집하여 'Feature Website.xlsx' 파일에 저장해 두었습니다.


### <span style="color:blue">[예시] Beatuifulsoup 라이브러리를 활용 HTML code를 출력하고 \<title> 태그 길이를 계산합니다.<span>

In [57]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(df['html_code'][0], 'html.parser')

*<span style="color:blue"> html code 출력<span>

In [58]:
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER"

* <span style="color:blue"> \<title> 태그 출력 및 길이 계산<span>

In [10]:
# <title> 태그 출력
print("* title :",soup.head.title)

# <title> 태그 길이 출력
print("* title 길이 :", len(str(soup.head.title.getText())))

* title : <title dir="ltr">Amazon.com</title>
* title 길이 : 10


---

## <b>Q2. html 에서 \<script>...\</script> 태그 길이 계산
- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제2] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<script> 태그 길이를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [63]:
def html_script_characters(soup):
    return float(sum(map(lambda x: len(str(x).replace(' ', ' ')) if x is not None else 0, list(soup.find_all('script')))))

In [65]:
script_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    script_len.append(html_script_characters(soup))
    
df['html_script_characters'] = script_len
df

,html_code,repu,html_script_characters
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,1127.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,993.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,10358.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,425.0
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,3197.0
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,24936.0
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,24089.0
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,1389.0
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,15086.0


In [44]:
# 추출한 Feature(특징) 데이터를 확인합니다.



## <b>Q3. html에서 공백 수 계산

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제3] Beatuifulsoup 라이브러리를 활용 HTML Code에서 \<html> 태그 공백 수를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [70]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def html_num_whitespace(soup):
    tag_string = str(soup)
    return sum([len(s) - len(s.lstrip()) for s in tag_string.split('\n')])
    

In [71]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.
whitespace_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    whitespace_len.append(html_num_whitespace(soup))
    
df['html_num_whitespace'] = whitespace_len
df

,html_code,repu,html_script_characters,html_num_whitespace
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,1127.0,225
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,993.0,102
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,10358.0,973
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,425.0,1825
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,3197.0,384
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,24936.0,467
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,24089.0,61
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,1389.0,288
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,15086.0,886


In [8]:
# 추출한 Feature(특징) 데이터를 확인합니다.


---

## <b>Q4. html 에서 body 길이 계산

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제4] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<body> 태그 길이를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [76]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def html_body_characters(soup):
    body = soup.body
    if body is None:
        return 0
    else:
        return len(body.text)
    

In [77]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.
body_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    body_len.append(html_body_characters(soup))
    
df['html_body_characters'] = script_len
df

,html_code,repu,html_script_characters,html_num_whitespace,html_body_characters
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,1127.0,225,1127.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,993.0,102,993.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,10358.0,973,10358.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,0,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,425.0,1825,425.0
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,3197.0,384,3197.0
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,24936.0,467,24936.0
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,24089.0,61,24089.0
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,1389.0,288,1389.0
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,15086.0,886,15086.0


In [14]:
# 추출한 Feature(특징) 데이터를 확인합니다.



---

## <b>Q5. script 에서 src, href 속성을 가진 태그수

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:darkred">[문제5] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<script> 태그에서 src, href 속성을 가진 태그수를 계산하는 함수를 완성하고 결과를 확인하세요. <span>


In [82]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
src_lengths = []
href_lengths = []
def calculate_tag_lengths(soup):
    for tag in soup.find_all():
        if tag.has_attr('src'):
            src_lengths.append(len(tag['src']))
        if tag.has_attr('href'):
            href_lengths.append(len(tag['href']))


In [84]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다
for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    lengths = calculate_tag_lengths(soup)
    src_lengths.append(lengths['src'])
    href_lengths.append(lengths['href'])
    
df['html_src_characters'] = src_lengths
df['html_href_characters'] = href_lengths
df

TypeError: 'NoneType' object is not subscriptable

In [1]:
# 추출한 Feature(특징) 데이터를 확인합니다.



## <b>Q6. 추가적으로 도출 가능한 Feature

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- 적절한 자료형으로 return 받기

### <span style="color:darkred">[문제6] Beatuifulsoup 라이브러리를 활용 HTML code에서 추가로 만들수 있는 Feature를 찾아보고 결과를 확인하세요. <span>


In [12]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.


In [ ]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다


In [ ]:
# 추출한 Feature(특징) 데이터를 확인합니다.

